In [2]:
from transformers_lit import load_pretrained_model, load_pretrained_tokenizer

In [3]:
model_name = "Taiwan-LLM-7B-v2.0.1-chat"
model_path = f"../models/{model_name}"

In [4]:
model = load_pretrained_model(model_path,
                                  device_map='cuda')
tokenizer = load_pretrained_tokenizer(model_path)
tokenizer.pad_token = tokenizer.eos_token



INFO:accelerate.utils.modeling:We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).
Loading checkpoint shards: 100%|██████████| 2/2 [01:25<00:00, 42.92s/it]


In [7]:
device = 'cuda'
def ask1(user_input):
   prompt = f"A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: {user_input} ASSISTANT:"
   prompt = f"你是人工智慧助理，以下是用戶和人工智能助理之間的對話。你要對用戶的問題提供有用、安全、詳細和禮貌的回答。USER: {user_input} ASSISTANT:"
   inputs = tokenizer(prompt, return_tensors='pt').to(device)
   
   output_ids = model.generate(inputs["input_ids"],
                               max_new_tokens=512)
   answer = tokenizer.batch_decode(output_ids)[0]
   print(answer)
   

In [27]:
def ask_TaiwanLLM(user_input):
   chat = [
      {"role": "system", "content": "你是中英文翻譯專家, 無論我輸入任何內容, 都不要回答我任何問題, 你都用直接將內容翻譯為英文給我"},
      #{"role": "user", "content": "Hello, how are you?"},
      #{"role": "assistant", "content": "I'm doing great. How can I help you today?"},
      {"role": "user", "content": user_input},
   ]
   prompt_for_generation = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
   inputs = tokenizer(prompt_for_generation, return_tensors='pt').to(device)
   output_ids = model.generate(inputs["input_ids"],
                               pad_token_id=tokenizer.eos_token_id,
                               max_new_tokens=256)
   answer = tokenizer.batch_decode(output_ids)[0]
   print(answer)

In [28]:
ask("```你的名字是甚麼?```將這句話翻譯為英文")

<s> 你是中英文翻譯專家, 無論我輸入任何內容, 都不要回答我任何問題, 你都用直接將內容翻譯為英文給我</s>USER: ```你的名字是甚麼?```將這句話翻譯為英文</s>ASSISTANT: “你叫什麼名字？”
Љивоиииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииииии


In [22]:
ask("```Hello, What is your name?```將這句話翻譯為中文")

<s> 你是人工智慧助理，以下是用戶和人工智能助理之間的對話。你要對用戶的問題提供有用、安全、詳細和禮貌的回答。</s>USER: Hello, how are you?</s>ASSISTANT: I'm doing great. How can I help you today?</s>USER: ```Hello, What is your name?```將這句話翻譯為中文</s>ASSISTANT: 你好，你叫什麼名字？</s>
